## Understanding Tweepy

In [1]:
# to interact with Twitter API
import tweepy
# for Authenticaion Keys
from os import getenv

In [2]:
# Load Environment Variables
TWITTER_API_KEY = getenv('TWITTER_API_KEY')
TWITTER_API_KEY_SECRET = getenv('TWITTER_API_KEY_SECRET')

In [14]:
auth = tweepy.OAuthHandler(TWITTER_API_KEY, TWITTER_API_KEY_SECRET)
TWITTER = tweepy.API(auth)
elon = TWITTER.get_user('elonmusk')

In [15]:
dir(elon)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_api',
 '_json',
 'contributors_enabled',
 'created_at',
 'default_profile',
 'default_profile_image',
 'description',
 'entities',
 'favourites_count',
 'follow',
 'follow_request_sent',
 'followers',
 'followers_count',
 'followers_ids',
 'following',
 'friends',
 'friends_count',
 'geo_enabled',
 'has_extended_profile',
 'id',
 'id_str',
 'is_translation_enabled',
 'is_translator',
 'lang',
 'listed_count',
 'lists',
 'lists_memberships',
 'lists_subscriptions',
 'location',
 'name',
 'notifications',
 'parse',
 'parse_list',
 'profile_background_color',
 'profile_background_image_url',
 'profile_back

In [16]:
dir(elon.status)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_api',
 '_json',
 'contributors',
 'coordinates',
 'created_at',
 'destroy',
 'entities',
 'favorite',
 'favorite_count',
 'favorited',
 'geo',
 'id',
 'id_str',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'is_quote_status',
 'lang',
 'parse',
 'parse_list',
 'place',
 'retweet',
 'retweet_count',
 'retweeted',
 'retweets',
 'source',
 'source_url',
 'text',
 'truncated']

In [17]:
elon.timeline()[0].text

'@PPathole After'

In [18]:
elon.timeline()[0].in_reply_to_screen_name

'PPathole'

In [19]:
elon.timeline()[0].retweeted

False

In [20]:
elon.timeline()[0].created_at

datetime.datetime(2020, 10, 15, 7, 53, 13)

In [59]:
# How to Get the Text of a Tweet
elon.timeline(
            count=200, exclude_replies=True, include_rts=False,
            tweet_mode='extended', since_id=None)[0].full_text

'The gauntlet has been thrown down! \n\nThe prophecy will be fulfilled. \n\nModel S price changes to $69,420 tonight!'

## Understanding the Analysis

In [41]:
"""SQLAlchemy models and utility functions for TwitOff."""
from flask_sqlalchemy import SQLAlchemy

DB = SQLAlchemy()


class User(DB.Model):
    """Twitter users corresponding to Tweets in the Tweet table."""
    id = DB.Column(DB.BigInteger, primary_key=True)
    name = DB.Column(DB.String(15), nullable=False)
    # Tweet IDs are ordinal ints, so can be used to fetch only more recent
    newest_tweet_id = DB.Column(DB.BigInteger)

    def __repr__(self):
        return '<User {}>'.format(self.name)


class Tweet(DB.Model):
    """Tweets and their embeddings from Basilica."""
    id = DB.Column(DB.BigInteger, primary_key=True)
    text = DB.Column(DB.Unicode(300))  # Allowing for full + links
    embedding = DB.Column(DB.PickleType, nullable=False)
    user_id = DB.Column(DB.BigInteger, DB.ForeignKey('user.id'), nullable=False)
    user = DB.relationship('User', backref=DB.backref('tweets', lazy=True))

    def __repr__(self):
        return '<Tweet {}>'.format(self.text)

In [52]:
nlp = spacy.load('en_core_web_sm')

In [62]:
def vectorize_tweet(tweet_text):
    return nlp(tweet_text).vector

In [65]:
# Example of Embedding
example_tweet = elon.timeline(
            count=200, exclude_replies=True, include_rts=False,
            tweet_mode='extended', since_id=None)[0].full_text
print(example_tweet)

vector = vectorize_tweet(example_tweet)
print('Shape', vector.shape)
print(vector)

The gauntlet has been thrown down! 

The prophecy will be fulfilled. 

Model S price changes to $69,420 tonight!
Shape (96,)
[ 9.1873957e-03 -1.0405062e+00 -2.4615030e-03 -7.8577274e-01
 -2.5952318e-01  1.5746862e-02  3.9108464e-01  1.4352538e-01
 -1.1414484e+00  5.7398647e-01 -4.4893992e-01  6.9922966e-01
 -2.1050255e-01  2.2449448e+00 -3.7715408e-01 -5.0495576e-02
  3.6779836e-01 -5.6217372e-01  3.5060742e-01  7.6106375e-01
  6.8657422e-01 -4.5199004e-01  5.3928733e-01 -5.2064949e-01
  6.7628354e-01  1.3119510e+00 -1.6024026e+00 -1.4628707e+00
 -7.1308875e-01 -1.3743941e+00  3.1411526e-01 -6.2586111e-01
  5.8948070e-01 -1.0099517e+00  2.4074154e-01  8.2665253e-01
 -6.8420881e-01 -2.9855013e-01 -6.6787273e-02 -4.6510974e-01
  7.9847807e-01  4.2252947e-02  6.9942760e-01 -8.4412092e-01
  5.7230085e-01 -6.3125056e-01 -3.9756756e-02  6.9487190e-01
  8.8051158e-01  3.6715022e-01  4.7167358e-01 -3.7116119e-01
  1.6800560e-01 -3.4775648e-01  3.4939387e-01  8.4622598e-01
  1.1168628e+00 -3.93

In [1]:
# Example of Embedding
example_tweet = elon.timeline(
            count=200, exclude_replies=True, include_rts=False,
            tweet_mode='extended', since_id=None)[1].full_text
print(example_tweet)

vector = vectorize_tweet(example_tweet)
print('Shape', vector.shape)
print(vector)

NameError: name 'elon' is not defined

In [44]:
def add_or_update_user(username):
    try:
        # Get User from Username from API
        twitter_user = TWITTER.get_user(username)
        # Create a Row in User Table
        db_user = User(id=twitter_user.id, name=username)
        
        # Get Tweets from Username from API
        tweets = twitter_user.timeline(
            count=200, exclude_replies=True, include_rts=False,
            # ?- What happens if db_user does not have a newest_tweet_id? -?
            tweet_mode='extended', since_id=db_user.newest_tweet_id)
        # Get newest_tweet_id
        if tweets:
            db_user.newest_tweet_id = tweets[0].id
        for tweet in tweets:
            embedding = vectorize_tweet(tweet.full_text)
            
            
    except Exception as e:
        print('Error processiing {}: {}'.format(username, e))
        raise e
        
    else:
        print("Commit Changes if in App")
        return db_user
    

In [45]:
mb = add_or_update_user('MorningBrew')

Commit Changes if in App


In [47]:
mb.newest_tweet_id

1316742064193900547